In [7]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-07 18:23:30--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-07 18:23:30 (45.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [9]:
!pip install openai -q


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [18]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [27]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [28]:
client

In [29]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag('write that this is test')

' To change from read-only mode to editable mode for dbt+git files, follow these steps:\n1. Create a new branch on GitHub by clicking on "New Branch" and naming it accordingly. For instance, name your branch as `edit_dbt_files`. This allows you to make changes without affecting the main codebase directly. Once created, switch back to this newly-created branch using Git commands:\n```git checkout -b edit_dbt_files```\n2. Now that we are in a new and dedicated environment for making edits on dbt+git files within our GitHub repository, any changes made here can be safely merged into the main codebase once completed without disrupting others\' work who might also need to modify some of these shared resources or assets independently - providing an added level of flexibility in a concurrent development setting.\n3. To make your desired edits and commit them back upstream, use standard Git workflow commands:\n```git add .\ngit commit -m "Editing dbt+Git files for clarity" \ngit push origin ed

In [31]:
print(_)

 To change from read-only mode to editable mode for dbt+git files, follow these steps:
1. Create a new branch on GitHub by clicking on "New Branch" and naming it accordingly. For instance, name your branch as `edit_dbt_files`. This allows you to make changes without affecting the main codebase directly. Once created, switch back to this newly-created branch using Git commands:
```git checkout -b edit_dbt_files```
2. Now that we are in a new and dedicated environment for making edits on dbt+git files within our GitHub repository, any changes made here can be safely merged into the main codebase once completed without disrupting others' work who might also need to modify some of these shared resources or assets independently - providing an added level of flexibility in a concurrent development setting.
3. To make your desired edits and commit them back upstream, use standard Git workflow commands:
```git add .
git commit -m "Editing dbt+Git files for clarity" 
git push origin edit_dbt_fi